In [11]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42


In [12]:
dataset = 'model/keypoint.csv'
model_save_path = 'model/keypoint_classifier.hdf5'
NUM_CLASSES = 3

In [13]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

In [20]:
X_train[0]

array([ 0.        ,  0.        , -0.2556391 , -0.04511278, -0.45864663,
       -0.22556391, -0.518797  , -0.45112783, -0.41353384, -0.60902256,
       -0.38345864, -0.6466165 , -0.43609023, -0.91729325, -0.35338345,
       -0.96240604, -0.2706767 , -0.86466163, -0.18796992, -0.66917294,
       -0.17293233, -1.        , -0.08270676, -0.93233085, -0.04511278,
       -0.7518797 , -0.01503759, -0.62406015,  0.03759398, -0.86466163,
        0.07518797, -0.7218045 ,  0.06766918, -0.5338346 ,  0.15037593,
       -0.5338346 ,  0.19548872, -0.65413535,  0.19548872, -0.5413534 ,
        0.15789473, -0.41353384], dtype=float32)

In [14]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [15]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_2 (Dropout)         (None, 42)                0         
                                                                 
 dense_3 (Dense)             (None, 20)                860       
                                                                 
 dropout_3 (Dropout)         (None, 20)                0         
                                                                 
 dense_4 (Dense)             (None, 10)                210       
                                                                 
 dense_5 (Dense)             (None, 3)                 33        
                                                                 
Total params: 1,103
Trainable params: 1,103
Non-trainable params: 0
_________________________________________________________________


In [16]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)

es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [17]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [18]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
 1/27 [>.............................] - ETA: 16s - loss: 1.2585 - accuracy: 0.3125
Epoch 1: saving model to model\keypoint_classifier.hdf5
27/27 [==============================] - 1s 10ms/step - loss: 1.1383 - accuracy: 0.3701 - val_loss: 1.0680 - val_accuracy: 0.4672
Epoch 2/1000
25/27 [==========================>...] - ETA: 0s - loss: 1.0660 - accuracy: 0.4294
Epoch 2: saving model to model\keypoint_classifier.hdf5
27/27 [==============================] - 0s 5ms/step - loss: 1.0649 - accuracy: 0.4297 - val_loss: 1.0061 - val_accuracy: 0.5517
Epoch 3/1000
24/27 [=========================>....] - ETA: 0s - loss: 1.0270 - accuracy: 0.4795
Epoch 3: saving model to model\keypoint_classifier.hdf5
27/27 [==============================] - 0s 5ms/step - loss: 1.0261 - accuracy: 0.4762 - val_loss: 0.9574 - val_accuracy: 0.7017
Epoch 4/1000
26/27 [===========================>..] - ETA: 0s - loss: 0.9897 - accuracy: 0.5021
Epoch 4: saving model to model\keypoint_classifier.hdf5
27/

In [19]:
model = tf.keras.models.load_model(model_save_path)